In [ ]:
import pandas as pd
import numpy as np
import os
from collections import Counter
from pprint import pprint
from datetime import datetime
from IPython.display import display
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import matplotlib.pyplot as plt

In [ ]:
os.chdir('d:\\Users\\Patri\\Documents\\1.TCC\\Notebooks\\WebScrapAcordaosPDF\\NLTK')

In [ ]:
%%time
df = pd.read_csv('Acordaos2020booleado.csv')

In [ ]:
# Colher amostra para conferir a rotulação
# from sklearn.model_selection import train_test_split
# amostra, restante = train_test_split(df, train_size = 10)
# amostra
# amostra.to_csv('amostra.csv')

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
# Criar coluna que indica se alguma seção do Acórdão além do Relatório teve Súmula aplicada
# df['S002TX'] = df[['S002VO', 'S002EM', 'S002AC', 'S002VV', 'S002DE']].max(axis=1)

cols=['VO', 'EM', 'AC', 'VV', 'DE']

for sum in range (1,162):
    lista_cols = []
    TX_col = 'S'+str(sum).zfill(3)+'TX'                      # 'S002TX'
    for AA in cols:
        S999AA = 'S'+str(sum).zfill(3)+AA
        lista_cols.append(S999AA)    # ['S002VO', 'S002EM', 'S002AC', 'S002VV', 'S002DE']
    df[TX_col] = df[lista_cols].max(axis=1)

In [ ]:
# Totaliza Súmula por seção
z_colunas=['VO', 'EM', 'RE', 'AC', 'VV', 'DE', 'TX']

dfT = pd.DataFrame(columns = z_colunas)

for sum in range (1,162):
    sumXXX = str(sum).zfill(3)
    linha = [ sumXXX ]
    for count, col in enumerate(z_colunas):    
        qtsum = int(df['S' + sumXXX + col].sum())
        linha.append(qtsum)
    # linha
    dflinha = pd.DataFrame([linha], columns = ['Sumula'] + z_colunas)
    dfT = dfT.append(dflinha)
    dfT = dfT.reset_index(drop = True)
dfT


In [ ]:
# Comparar uso da Súmula no Voto com uso da Súmula em todas as seções exceto o Relatório
cols=['VO', 'EM', 'AC', 'VV', 'DE']
for sum in range (1,162):
    lista_cols = []
    TX_col = 'S'+str(sum).zfill(3)+'TX'                      # 'S002TX'
    for AA in cols:
        S999AA = 'S'+str(sum).zfill(3)+AA
        lista_cols.append(S999AA)    # ['S002VO', 'S002EM', 'S002AC', 'S002VV', 'S002DE']
    

In [ ]:
dfT['VO_TX'] = (dfT['TX']-dfT['VO'])
dfT

In [ ]:
dfT[['VO', 'EM', 'RE', 'AC', 'VV', 'DE', 'TX', 'VO_TX']].sum()

In [ ]:
df = df.fillna('')
df = df.replace(np.nan,'')

In [ ]:
df['s_Relatorio']

In [ ]:
# Totalizar as súmulas aplicadas por coluna
colunas=['Voto', 'Ementa', 'Relatorio', 'Acordao', 'VotoVencedor', 'DeclaracaoVoto']

sum_total = []
for count, coluna in enumerate(colunas): 
    sum_coluna = []
    for index, values in df['s_'+coluna].items():
        if values: 
            lista = values.strip('][').split(', ')
            for sumula in lista:
                 if len(sumula):
                    sum_coluna.append(int(sumula))
                    sum_total.append(int(sumula))
        sum_coluna = sorted(list(set(sum_coluna)))    
    print ("Coluna:", coluna, len(sum_coluna), sum_coluna)
sum_total = sorted(list(set(sum_total)))
print ("Total:", len(sum_total), sum_total)


In [ ]:
dfT = dfT.set_index('Sumula')

In [ ]:
dfT['VO_percent'] = 100 * dfT['VO']/dfT['VO'].sum()

In [ ]:
dfT[['VO', 'VO_percent']].sort_values('VO').query("Sumula == ['137', '128', '017', '157', '108', '001', '049', '002']")

In [ ]:
dfT[['VO', 'VO_percent']].sort_values('VO')

In [ ]:
pd.to_numeric(dfT['VO'], downcast='float').describe()

In [ ]:
dfT['VO'].sort_values().plot(kind='area')

In [ ]:
# Remover as colunas de textos que não serão utilizadas para a predição
colunas = ['Ementa', 'Acordao', 'Voto', 'VotoVencedor', 'DeclaracaoVoto']
s_colunas = list('s_' + coluna for coluna in colunas)
remover_colunas = s_colunas
remover_colunas

for coluna in remover_colunas:
    df = df.drop(columns=coluna)

df

In [ ]:
%%time
df.to_csv('Acordaos2020tx.csv', index = False)